### Setup

In [ ]:
from manim import *

config.media_width = "75%"
config.verbosity = "WARNING"

# Manim's building blocks

Manim offers three fundamental classes: `MObject`, `Animation` and `Scene`.

## Mobjects

- Each class derived from `MObject` (mathematical object) represents an object that can be displayed from screen.
- Since `Mobject` is an abstract base class, you will most likely create instances of its derived classes.

### Creating and displaying MObjects

- Usually all of the code in a manim script is put inside the `construct()` method of a `Scene` class.
- Call the `add()` method of the containing `Scene` to display a mobject on the screen. This does not animate the process.
- Remove a mobject on the screen using `remove()` method.

In [ ]:
%%manim -qh CreatingMobjects

class CreatingMobjects(Scene):
    def construct(self):
        circle = Circle()
        self.add(circle)
        self.wait(1)
        self.remove(circle)
        self.wait(1)

### Placing mobjects

- By default, mobjects are placed at the center of coordinates, or origin.
- Manim places the center of coordinates at the center of the screen.
- They are also given default colors.
- Move a mobject using `shift()` method.

In [ ]:
%%manim -qh Shapes

class Shapes(Scene):
    def construct(self):
        circle = Circle()
        square = Square()
        triangle = Triangle()

        # Move each shape one unit from the origin given a vector
        circle.shift(LEFT)
        square.shift(UP)
        triangle.shift(RIGHT)

        self.add(circle, square, triangle)
        self.wait(1)

- Use `move_to()` to move mobjects using absolute units (measured from the `ORIGIN`).
- Use `next_to()` to move mobjects using relative units (measured from the mobject passed as the first argument).
- Use `align_to()` to align mobject to another `Mobject` in a certain direction.

In [ ]:
%%manim -qh MobjectPlacement

class MobjectPlacement(Scene):
    def construct(self):
        circle = Circle()
        square = Square()
        triangle = Triangle()

        # Place the circle two units left from the origin
        circle.move_to(LEFT * 2)
        # Place the square to the left of the circle
        square.next_to(circle, LEFT)
        # Align the left border of the triangle to the left border of the circle
        triangle.align_to(circle, LEFT)

        self.add(circle, square, triangle)
        self.wait(1)

- The coordinates of the borders of a mobject are determined using an imaginary bounding box around it.
- Use `SurroundingRectangle` to show the boudning box of a mobject.

In [ ]:
%%manim -qh BoundingBoxExample

class BoundingBoxExample(Scene):
    def construct(self):
        circle = Circle()
        self.add(circle)

        # Create a bounding box around the circle
        bounding_box = SurroundingRectangle(circle)

        # Display the bounding box
        self.add(bounding_box)
        self.wait(1)

### Styling mobjects

- Use `set_stroke()` method to change the visual style of a mobject's border.
- Use `set_fill()` method to change the visual style of the interior.
- By default, most mobjects have a fully transparent interior. Specify the opacity parameter to display the color.

In [ ]:
%%manim -qh MobjectStyling

class MobjectStyling(Scene):
    def construct(self):
        circle = Circle().shift(LEFT)
        square = Square().shift(UP)
        triangle = Triangle().shift(RIGHT)

        circle.set_stroke(color=GREEN, width=20)
        square.set_fill(YELLOW, opacity=1.0)
        triangle.set_fill(PINK, opacity=0.5)

        self.add(circle, square, triangle)
        self.wait(1)

### Mobject on-screen order

- The order of the arguments of `add()` determines the order that the mobjects are displayed on the screen, with the left-most arguments being put in the back.

In [ ]:
%%manim -qh MobjectZOrder

class MobjectZOrder(Scene):
    def construct(self):
        circle = Circle().shift(LEFT)
        square = Square().shift(UP)
        triangle = Triangle().shift(RIGHT)

        circle.set_stroke(color=GREEN, width=20)
        square.set_fill(YELLOW, opacity=1.0)
        triangle.set_fill(PINK, opacity=0.5)

        # The current scene is same as the previous `MobjectStyling` scene,
        # except for the following line. Notice the order of the arguments.
        self.add(triangle, square, circle)
        
        self.wait(1)

## Animations

- Animations are procedures that interpolate between two mobjects.
- In this context, interpolation is inbetweening, or filling in frames between two mobjects. 
- Add animation to your scene using `play()` method.

In [ ]:
%%manim -qh SomeAnimations

class SomeAnimations(Scene):
    def construct(self):
        square = Square()

        # Some animations display mobjects
        self.play(FadeIn(square))

        # Some move or rotate mobjects around
        self.play(Rotate(square, PI/4))

        # Some animations remove mobjects from the screen
        self.play(FadeOut(square))

        self.wait(1)

### Animating methods

- Any property of a mobject that can be changed can be animated.
- Prepending `animate()` property to a method that changes a mobject’s property animates that method.

In [ ]:
%%manim -qh AnimateExample

class AnimateExample(Scene):
    def construct(self):
        square = Square().set_fill(RED, opacity=1.0)
        self.add(square)

        # Animate the change of color
        self.play(square.animate.set_fill(WHITE))
        self.wait(1)

        # Animate the change of position and the rotation at the same time
        self.play(square.animate.shift(UP).rotate(PI / 3))
        self.wait(1)

### Animation run time

- By default, any animation passed to `play()` lasts for exactly one second. 
- Use the `run_time` argument to control the duration.

In [ ]:
%%manim -qh RunTime

class RunTime(Scene):
    def construct(self):
        square = Square()
        self.add(square)
        self.play(square.animate.shift(UP), run_time=3)
        self.wait(1)

### Creating custom animation

- Manim provides built-in animations, but there are times when custom animations are needed.
- To create custom animations, extend the `Animation` class and override the `interpolate_mobject()` method.
- The `interpolate_mobject()` method receives an `alpha` parameter that ranges from 0 to 1 throughout the animation.
- Manipulate `self.mobject` inside `interpolate_mobject()` based on the `alpha` value to achieve the desired animation effect.
- Custom animations benefit from features such as different run times and rate functions.

#### Example: Counting animation

For transforming a number smoothly, creating a custom animation is intuitive.
- Create a `Count` class that extends `Animation`, taking a `DecimalNumber` mobject, `start`, and `end` as arguments.
- Pass the `DecimalNumber` mobject to the super constructor and set the `start` and `end` values in the `Count` constructor.
- In the `interpolate_mobject()` method, determine the number to display at each step based on the alpha value. Use the logic: `starting_number + alpha * (target_number - starting_number)`.
- Use the `set_value()` method of the `DecimalNumber` mobject to update its displayed number.
- Once the `Count` animation is defined, it can be played in a `Scene` class with different durations and rate functions.

In [ ]:
%%manim -qh CountingScene

class Count(Animation):
    def __init__(
        self, number: DecimalNumber, start: float, end: float, **kwargs
    ) -> None:
        # Pass number as the mobject of the animation
        super().__init__(number, **kwargs)
        # Set start and end
        self.start = start
        self.end = end

    def interpolate_mobject(self, alpha: float) -> None:
        # Set value of DecimalNumber according to alpha
        value = self.start + (alpha * (self.end - self.start))
        # `self.mobject` refers to the `DecimalNumber` object being animated
        self.mobject.set_value(value)


class CountingScene(Scene):
    def construct(self):
        # Create Decimal Number and add it to scene
        number = DecimalNumber().set_color(WHITE).scale(5)
        # Add an updater to keep the DecimalNumber centered as its value changes
        number.add_updater(lambda number: number.move_to(ORIGIN))

        self.add(number)

        self.wait()

        # Play the Count Animation to count from 0 to 100 in 4 seconds
        self.play(Count(number, 0, 100), run_time=4, rate_func=linear)

        self.wait()

### Using coordinates of a mobject

- Mobjects contain points that define their boundaries. 
- These points can be used to add other mobjects respectively to each other.

In [ ]:
%%manim -qh MobjectExample

class MobjectExample(Scene):
    def construct(self):
        p1= [-1,-1,0]
        p2= [1,-1,0]
        p3= [1,1,0]
        p4= [-1,1,0]
        a = Line(p1,p2).append_points(Line(p2,p3).points).append_points(Line(p3,p4).points)
        point_start= a.get_start()
        point_end  = a.get_end()
        point_center = a.get_center()
        self.add(Text(f"a.get_start() = {np.round(point_start,2).tolist()}", font_size=24).to_edge(UR).set_color(YELLOW))
        self.add(Text(f"a.get_end() = {np.round(point_end,2).tolist()}", font_size=24).next_to(self.mobjects[-1],DOWN).set_color(RED))
        self.add(Text(f"a.get_center() = {np.round(point_center,2).tolist()}", font_size=24).next_to(self.mobjects[-1],DOWN).set_color(BLUE))

        self.add(Dot(a.get_start()).set_color(YELLOW).scale(2))
        self.add(Dot(a.get_end()).set_color(RED).scale(2))
        self.add(Dot(a.get_top()).set_color(GREEN_A).scale(2))
        self.add(Dot(a.get_bottom()).set_color(GREEN_D).scale(2))
        self.add(Dot(a.get_center()).set_color(BLUE).scale(2))
        self.add(Dot(a.point_from_proportion(0.5)).set_color(ORANGE).scale(2))
        self.add(*[Dot(x) for x in a.points])
        self.add(a)

### Transforming mobjects into other mobjects

In [ ]:
%%manim -qh ExampleTransform

class ExampleTransform(Scene):
    def construct(self):
        self.camera.background_color = WHITE
        m1 = Square().set_color(RED)
        m2 = Rectangle().set_color(RED).rotate(0.2)
        self.play(Transform(m1,m2))

In [ ]:
%%manim -qh ExampleRotation

class ExampleRotation(Scene):
    def construct(self):
        self.camera.background_color = WHITE
        m1a = Square().set_color(RED).shift(LEFT)
        m1b = Circle().set_color(RED).shift(LEFT)
        m2a= Square().set_color(BLUE).shift(RIGHT)
        m2b= Circle().set_color(BLUE).shift(RIGHT)

        points = m2a.points
        points = np.roll(points, int(len(points)/4), axis=0)
        m2a.points = points

        self.play(Transform(m1a,m1b),Transform(m2a,m2b), run_time=1)

## Scenes

- The `Scene` class is crucial in manim as it serves as the core component that connects various elements.
- Mobjects need to be added to a scene to be displayed and removed from it to stop being displayed.
- Animations must be played by a scene, and intervals without animations are determined using the `wait()` function.
- All the code for a video should be placed within the `construct()` method of a class derived from `Scene`.
- It is possible to have multiple `Scene` subclasses within a single file if multiple scenes need to be rendered simultaneously.